In [ ]:
%load_ext autoreload
%autoreload 2

import visualizations as vis
import support as sp

# Starter Kit 4.1: Intelligent data retention: time series compression

## Passport

### Business context

More and more assets are instrumented and connected, in order to collect usage and performance data such that
they can be monitored (semi-)continuously. A large number of sensors can be installed which log data in high frequency, resulting in a large amount of data being collected. 

Typically, these large volumes of data are transferred to a central server, where they are further analysed. This is not always optimal however:
- a stable communication mechanism is required which can be very expensive or is sometimes simply not available. A vehicle operating in a remote area for example doesn't always have a decent connection available, and sending huge amounts of data through a satellite connection is quite expensive.
- if a continuous communication mechanism is not guaranteed, the data is typically stored locally first until it can be offloaded when communication can be established. However, providing storage also has an associated cost and if data is not offloaded periodically, storage will run out and parts of the data will need to be overwritten.
- when data can be offloaded, e.g. a vehicle is close to a hotspot with a communication mechanism, this has to happen in a limited amount of time. Requiring a vehicle to stay with the sole purpose of offloading data is not acceptable. As such, it is not always possible to offload all data at once. 

In such kind of situations, data compression can be applied in order to reduce the size of the data that needs to be stored or transferred. Typically, lossless data compression techniques (such as zip) are considered, but these are suboptimal: not only are they resource demanding, they also consider each data point as equally important. However, since assets rarely fail, data representing _normal_ behaviour is available in abundant quantities, while data representing _abnormal_ behaviour is rather rare. In most cases, the abnormal data is much more interesting to retain at a high level of detail, as a thorough analysis of the abnormality is typically performed in order to determine the root cause. 

More intelligent data retention methods are able to distinguish between high and low informative data, i.e. abnormal and normal usage data, and can compress the data with low informativeness to a higher degree than the data with high informativeness. 

### Business goal

The business goal related to this Starter Kit is **intelligent data retention**. More specifically, we will present a technique called "Swinging Door Trending" [1] that considers the informativeness of the data in order to retain at high frequency the interesting data while reducing the frequency of the less interesting data, thereby optimally using the storage or communication means of the asset.  

The level of informativeness of the data is typically linked to the application for which the data is used. In this Starter Kit, an event detection algorithm will be used to validate the effectiveness of the technique. An intelligent data retention technique should effectively retain the most important data points such that the results of the application, in this case event detection, will be the same or only slightly different at most. 


### Application context

- Professional vehicles, such as cranes, trucks or airplanes, operating in remote areas without high-speed communication means
- Wearable devices with a limited amount of memory and computing power that need to transfer data to secondary devices (e.g. a mobile phone)
- Mobile industrial assets, such as compressors, that are leased and typically not connected to a company's internal internet infrastructure for safety reasons
- ...



### Starter Kit outline

We will first describe the dataset that we will be using for this Starter Kit, which consists of accelerometer data containing six time series signals. Then, we conceptually explain the Swinging Door Trending technique, and apply this on the six signals separately. Afterwards, we apply the technique on the six signals at once, which makes sense since accelerometer signals are often correlated. We then evaluate the results of both compression approaches on a technical level by considering how much they manage to compress the original signals and what is the associated error. Finally, we validate the compression within a concrete application, i.e. event detection. 

## Dataset

The dataset used in this Starter Kit concerns over 100 drives of a vehicle that stores usage data while driving. The vehicle does not have a continuous communication means and thus cannot send the usage data to the centralized server continuously. Data can only be transferred when the vehicle reaches a location where a hotspot is installed to off-load the data. In this case, the hotspot is located at the entry of a parking spot, so the amount of time the vehicle is in range of the hostpot is limited, resulting in small amounts of data that can be off-loaded at once. 

The dataset consists of accelerometer data that is aggregated:
the raw data is sampled at 200 Hz and two aggregations are calculated and retained each second,
the **min** and **max** value.
This results in six signals, identified as **minXAcc, maxXAcc, minYAcc, maxYAcc, minZAcc** and **maxZAcc** at 1 Hz, as shown in the table below.

In [ ]:
df = sp.load_data()
df.sample(20, random_state=1)

## Swinging Door Trending

In this Starter Kit we take a closer look at one particular state-of-the-art technique for intelligent data retention, called _Swinging Door Trending_ (or SDT). Before applying it on our dataset, we first provide an  explanation of how it works conceptually below.

The SDT technique compresses a sequence of data points by using a simplified linear representation. It is computationally simple, so it can run on low-resource devices without significant overhead. 

The technique considers an error deviation parameter $\Delta y$, which determines the error between the original signal and the compressed signal that one is willing to tolerate. The technique passes through the time series data sequentially, starting with the first two points. The first point is retained, a line is drawn between this point and the second point, and a so-called _tolerance band_ is defined by computing the upper and lower bound, based on the deviation parameter. 

An iterative process is started, where the technique considers the next point in the time series and verifies whether that point falls within the tolerance band. If this is the case, the point does not need to be retained and the next data point is considered. This continues until a point is reached that does not fall within the tolerance band, meaning this point cannot be represented by a linear representation between the point last retained and the current point. This last point is then also retained, and a new iterative process starts. 

At each step, the tolerance band is also updated. When a next point is considered, its upper and lower bounds are reconsidered. The upper or lower bounds will be updated if the bounds of the next point fall below or above the previous ones, respectively, i.e. if the upper bound of the new point is lower than the previous upper bound, the upper bound is updated, and vice versa for the lower bound.  This ensures that the tolerance band will always become smaller when new points are added, since the more points, the less precise the linear representation will become.

The process can be illustrated by means of an example as follows.
- In a first phase, the technique starts with the first point in the time series, the potential tolerance band starts with the whole area after the point. When the second point is considered, a linear interpolation is constructed between these two points and the tolerance band is updated using the deviation parameter $\Delta y$. 

<table><tr><td><img src='media/SDTPhase1.png'></td><td><img src='media/SDTPhase2.png'></td></tr></table>

- In a second phase, a next point is selected, the new linear representation is constructed between the last point that was retained and the next point, and the new tolerance band is defined. If the next point, i.e. point 4, has the new lower bound lower than the previous lower bound, the tolerance band is not updated (the purple area will not be used as a tolerance band update), similar for the upper bound. 

<table><tr><td><img src='media/SDTPhase3.png'></td><td><img src='media/SDTPhase4.png'></td></tr></table>

- As long as there are new points that fall in the tolerance band, they can be represented by the linear representation. Once a new point is considered that falls outside the tolerance band, e.g. point 6, the last point of the linear representation, e.g. point 5, will be retained as the compressed version of points 1 to 5. Starting from point 5 a new linear representation will be constructed with point 6 as the first point to consider.  

<table><tr><td><img src='media/SDTPhase5.png'></td><td><img src='media/SDTPhase6.png'></td></tr></table>

In this example, the first 5 points will be compressed by a linear representation that is defined by point 1 and point 5, i.e. removing points 2, 3 and 4.

## Applying SDT

In the next sections, we describe two different strategies for applying the SDT technique on the accelerometer data:
(i) by considering each of the six accelerometer signals separately and
(ii) by considering all of the signals combined.
The motivation for compressing the signals in a combined way is that accelerometer data is captured using a single sensor, hence, informativeness of the accelerometer measurements might be viewed as a whole using the x, y and z axes simultaneously.

### Strategy for setting error deviation parameter

In order to apply the SDT technique on our dataset of vehicle drives, we need to consider an appropriate value for the error deviation parameter, i.e. the error we are willing to tolerate and that influences the amount of compression that can be applied. This parameter can be set by a domain expert that knows the data well, or alternatively can be derived from the data, as we will do here.  For this purpose, we will use the **Self-definition SDT (SSDT)** adaptation of the original SDT algorithm.

Our dataset consists of different drives and the context of each drive can be different, e.g. the vehicle can drive over a highway or off road over rough terrain. This will obviously influence the accelerometer data and therefore it doesn't make sense to consider a fixed value for the error deviation parameter across the whole dataset. Rather, we will define the value for each drive separately, so that the data for each drive can be compressed optimally. 

In order to do this, we compute the mean of the variance of several subsets from a drive, i.e., 1000 random sized subsets of the drive are extracted and the mean of all their variances is calculated. This gives us a variance threshold that represents a rough estimate of when the datapoints are very informative.

### Seperate signal compression

As described above, the dataset consists of six signals per drive, identified as **minXAcc**, **maxXAcc**, **minYAcc**, **maxYAcc**, **minZAcc** and **maxZAcc**. We will first consider each of these signals as a separate time series and compress them individually.

In [ ]:
df_compressed_individually = sp.separate_compression(df)

We illustrate the results of the compression by plotting the original **maxXAcc** signal (blue line in the plot below) next to its compressed signal (orange line). The compression is applied per drive, so we select 4 representative drives out of more than 100 for visualisation.

In [ ]:
vis.plot_grouped(df, df_compressed_individually, 'drive', signal='maxXAcc', groups=[13, 25, 42, 43])

In the plot above, we can notice that the compression handles the variation of different drives very well. In drive 43 (the last plot), the first data points do not contain a high amount of informativeness and are therefore very well compressed, while the later part of the signal contains more information and is therefore less compressed. 

This indicates that deriving the value for the deviation parameter from the data actually works. But in some cases, e.g. drive 25, the variance of the signal is very low during the whole drive, which results in a very small deviation parameter. Consequently, the drive is hardly compressed, while the informativeness of the data is very low. This issue can be resolved by adding an extra check based on domain knowledge, e.g. a minimum variance or standard deviation threshold to consider when compressing a drive.

In order to confirm the results from the selected drives, we also select 10 random drives and visualise the results of applying the technique. 

In [ ]:
vis.plot_grouped(df, df_compressed_individually, group='drive', signal='maxXAcc', random_drives=True)

Notice that in some cases, only the first point is retained, meaning that the whole drive can be compressed as it does not contain enough informativeness and can be ommited in full.

### Combined signal compression method

Next, we will compress the different accelerometer signals simultaneously, i.e., the compression happens throughout the signals together. Each signal is processed sequentially and the decision to compress the points of all signals depends on whether or not the points can be represented by the linear representation of each dimension. For example, if the current point in signal A can be compressed by the linear representation for A **and** the current point in signal B can be represented by the linear representation for B, both points will be compressed. If the current point in signal A or B cannot be represented in their respective linear representation, both points will **not** be compressed.

As described earlier, min and max aggregations are calculated every second on the three axes of the raw data. We will view the min aggregated signals and the max aggregated signals separately, i.e. the SDT technique will be applied on the **minXAcc**, **minYAcc** and **minZAcc** signals combined, as well as on the **maxXAcc**, **maxYAcc** and **maxZAcc** signals combined.

In [ ]:
df_compressed_combined = sp.combined_compression(df)

The resulting signals can be plotted with respect to the original signal. As an example, the **maxXAcc** signal of 4 drives is plotted. 

In [ ]:
vis.plot_grouped(df, df_compressed_combined, group='drive', signal='maxXAcc', groups=[13, 25, 42, 43])

The results for this approach resemble the results of the previous approach. It does well to differentiate the high informative parts of the signal and the low informative sections, e.g. drive 6 and 9. In drive 21, the same remarks as previously apply: when the variance of the time series is low, the technique will calculate a low deviation parameter, resulting in a low compression of a signal that is probably just white noise.

Again 10 random drives are plotted to validate the results. For comparison reasons, these are the same as before.

In [ ]:
vis.plot_grouped(df, df_compressed_combined, group='drive', signal='maxXAcc', random_drives=True)

## Technical evaluation

In this section, we evaluate the compression that is achieved by comparing different characteristics, i.e. compression ratio, file size and root mean square error, of the compressed datasets and the original dataset. 

### Compression ratio
The compression ratio defines the amount of compression that is achieved when compressing a dataset. This is achieved by considering the number of data points in each dataset, and dividing the number of data points of the original dataset by the number of data points in the compressed datasets.

In [ ]:
sp.compression_ratio(df, df_compressed_individually, df_compressed_combined)

As can be seen, sequential compression achieves a compression rate of around 5, while the combined compression achieves a rate of 5.6. This indicates that the compressed datasets contain 5 and 5.6 times less data points than the original dataset, respectively. 

### File size
Another way to evaluate the compression is to check the file size when the data is saved as a file, e.g. a CSV file. The content of the datasets is saved using the same types, i.e. all numbers are converted to a standard format, float32.

In [ ]:
sp.file_size(df, df_compressed_individually, df_compressed_combined)

The size of the stored CSV file of both compressed datasets is almost the same.

Notice that there is a larger difference between the CSV file with the original dataset and the CSV files with the compressed versions, as the latter are stored in a file that are 7.8 and 6.2 times smaller respectively, not 5 as the compression ratio would suggest. This might be due to technical details, e.g. the performance of the CSV file writer of the data processing library we are using.

### Signal error
The difference between two signals is typically characterized by a metric called **root mean squared error (RMSE)**. Another way of evaluating the compression is thus to calculate this RMSE between the original signal and the two compressed signals.

However, we do need to take extra care: the compressed signals are smaller, since they are compressed, so comparing the time series is not as straightforward as it might seem. In order to address this, the two compressed signals are mapped onto the original signals, using the original timestamps. This means some missing values are introduced in the compressed signals, i.e. at those time points where data points were discarded. These missing values are imputed via a linear interpolation between the last know value and the next known value as the compression algorithm represents the discarded points by a linear representation between two points. 

In [ ]:
sp.rmse(df, df_compressed_individually, df_compressed_combined)

The RMSE for both compressed signals is quite low, which means there is little error between the original signal and the compressed signals, confirming the initial assumption that applying the SDT algorithm indeed leads to a very small loss in informativeness.

## Validation within an application

Next to validating whether the data is indeed compressed and to what extent, we should also validate whether the informativeness of the data is retained at a sufficiently high level. We aim at retaining the most informative segments of the signal, also after compression.

As discussed above, the level of informativeness of a dataset is typically linked to the application for which that dataset is used. In what follows, we will consider an event detection approach and validate whether the results of the application are influenced by applying the SDT technique. We apply the approach on the original dataset and on the compressed datasets and compare the results.

### Event detection

We use a simple event detection approach that defines a threshold for very high/low accelerometer values and flags data points above/below that threshold as events. We also assign a severity to the event, based on the percentage of how much the accelerometer value is exceeding the threshold. 

The threshold values that we use are:
- X-axis acceleration: min < -2.5; max > 1.5
- Y-axis acceleration: min < -1.8; max > 1.3
- Z-axis acceleration: min < -2.5; max > 2  


and we apply the following rules to define the severity:
- WARNING: if accelerometer value is between 100% and 150% of the threshold value
- INCIDENT: if accelerometer value is between 150% and 200% of the threshold value
- HARD INCIDENT: if accelerometer value is above 200% of the threshold value

Next, we apply this approach to all datasets.

In [ ]:
y_original, y_compressed_individually, y_compressed_combined, event_labels = sp.event_detection(df, df_compressed_individually, df_compressed_combined)

We then compare each of the events identified in the compressed dataset to the events found in the original dataset, in order to validate whether information loss occurred. 

Result detection of events on original signal:

In [ ]:
y_original

In [ ]:
vis.plot_confusion_matrix(y_original, y_compressed_individually, y_compressed_combined, event_labels, df)

As we can see, both the sequential and combined compressed datasets retain the required level of informativeness for this validation as they find 100% of the events for each class.
Both datasets find 10 warnings, 1 incident,  2 hard incidents and the rest are no events. The misclassifications are all 0% so no false positives are found.

## Conclusion

Data retention techniques aim to compress data while trying to retain a high level of informativeness. In this Starter Kit, we presented one such technique, Swinging Door Trending, that compresses data points through the use of a linear trend line. 

We used the technique to compress accelerometer data captured by a vehicle in two ways. First, we applied it sequentially to the 6 different signals, i.e. minimum and maximum of x-, y- and z-axis accelerations, and merged those compressed signals into a single compressed dataset. Second, we used it to compress the min and max versions of all three signals at the same time. 

We performed a technical evaluation of the compression, showing that the technique was able to compress the data with at least a factor 5, and validated whether the informativeness of the data is retained at a sufficiently high level in the context of a concrete application, i.e. event detection.
This confirmed that there is no loss of informativeness, as we detected the same events for the sequentially compressed dataset and for the combined compressed dataset as there where in the original dataset.

A main attention point is that the performance of this techniques relies on the correct definition of the deviation parameter. In this Starter Kit, the parameter was defined automatically and it performed very well, while in some other cases, this parameter needs to be defined using more contextual information to guarantee a good performance. 

## Bibliography

[1] Correa, Juan David Arias, et al. "Swinging door trending compression algorithm for iot environments." Anais Estendidos do IX Simpósio Brasileiro de Engenharia de Sistemas Computacionais. SBC, 2019.

## Additional information
Copyright © 2022 Sirris

This Starter Kit was developed in the context of the EluciDATA project (http://www.elucidata.be). For more information, please contact info@elucidata.be.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Notebook"), to deal in the Notebook without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Notebook, and to permit persons to whom the Notebook is provided to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies of the Notebook and/or copies of substantial portions of the Notebook.

THE NOTEBOOK IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL SIRRIS, THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, DIRECT OR INDIRECT, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE NOTEBOOK OR THE USE OR OTHER DEALINGS IN THE NOTEBOOK.